In [74]:
#run in python 2.7, not upgraded to 3.x
#script for just one sensor data

import arable
from arable.client import ArableClient
from datetime import datetime
import pandas as pd
from io import StringIO
import sensor_includes

In [75]:
a = ArableClient()
a.connect(email, password = password_ccber, tenant = tenant_ccber)

In [31]:
device = ['A000563']

In [83]:
l = a.devices()
device_l = []
for s in l:
    for i in s.values():
        if str(i).startswith('A000'):
            device_l.append(str(i))
print device_l

['A000513', 'A000502', 'A000507', 'A000465', 'A000546', 'A000369', 'A000229', 'A000522', 'A000536', 'A000466', 'A000318', 'A000563', 'A000514', 'A000417', 'A000461', 'A000543']


In [17]:
sta = "2019-01-01 00:00:00"
end = datetime.now()
end = end.strftime("%Y-%m-%dT%H:%M:%SZ")

In [18]:
df = a.query(select = 'all',
            format = 'csv',
            devices = device,
            measure = "hourly", #daily
            order = "time",
            end = end,
            start = sta,
            limit = 100000000)

In [19]:
df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)
df = df.drop('location', 1)
df.to_csv('A000563.csv', sep = ",")